# Reinforcement Learning Fundamentals
## Interactive Demo: Q-Learning Implementation

This notebook demonstrates the core concepts of Reinforcement Learning through a hands-on Q-Learning implementation.

### Learning Objectives:
- Understand the RL framework (Agent, Environment, State, Action, Reward)
- Implement Q-Learning algorithm from scratch
- Visualize the learning process
- Explore exploration vs exploitation trade-offs

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap
import pandas as pd
from IPython.display import clear_output
import time

# Set style for better visualizations
plt.style.use('default')
sns.set_palette("husl")

print("✅ Libraries imported successfully!")
print("🎯 Ready to explore Reinforcement Learning!")

## 1. Grid World Environment

We'll create a simple grid world where an agent learns to navigate from a start position to a goal while avoiding obstacles.

In [ ]:
class GridWorld:
    def __init__(self, size=5, obstacles=None, goal=None, start=None):
        self.size = size
        self.grid = np.zeros((size, size))
        
        # Set default positions
        self.start = start if start else (0, 0)
        self.goal = goal if goal else (size-1, size-1)
        self.obstacles = obstacles if obstacles else [(2, 2), (1, 3), (3, 1)]
        
        # Mark obstacles (-1), goal (1), and empty spaces (0)
        for obs in self.obstacles:
            if 0 <= obs[0] < size and 0 <= obs[1] < size:
                self.grid[obs] = -1
        self.grid[self.goal] = 1
        
        # Current position
        self.current_pos = self.start
        
        # Actions: 0=up, 1=right, 2=down, 3=left
        self.actions = [(0, -1), (1, 0), (0, 1), (-1, 0)]
        self.action_names = ['Up', 'Right', 'Down', 'Left']
    
    def reset(self):
        """Reset environment to start position"""
        self.current_pos = self.start
        return self.current_pos
    
    def step(self, action):
        """Take action and return (next_state, reward, done)"""
        # Calculate new position
        new_pos = (
            self.current_pos[0] + self.actions[action][0],
            self.current_pos[1] + self.actions[action][1]
        )
        
        # Check boundaries
        if (new_pos[0] < 0 or new_pos[0] >= self.size or 
            new_pos[1] < 0 or new_pos[1] >= self.size):
            return self.current_pos, -0.1, False
        
        # Check obstacles
        if new_pos in self.obstacles:
            return self.current_pos, -1.0, False
        
        # Move to new position
        self.current_pos = new_pos
        
        # Check if reached goal
        if self.current_pos == self.goal:
            return self.current_pos, 10.0, True
        
        return self.current_pos, -0.01, False
    
    def visualize(self):
        """Visualize the grid world"""
        plt.figure(figsize=(8, 6))
        vis_grid = self.grid.copy()
        vis_grid[self.current_pos] = 0.5
        
        plt.imshow(vis_grid, cmap='RdYlGn', vmin=-1, vmax=1)
        plt.title('Grid World Environment')
        
        # Add annotations
        for i in range(self.size):
            for j in range(self.size):
                if (i, j) == self.start:
                    plt.text(j, i, 'S', ha='center', va='center', fontsize=12, fontweight='bold')
                elif (i, j) == self.goal:
                    plt.text(j, i, 'G', ha='center', va='center', fontsize=12, fontweight='bold')
                elif (i, j) in self.obstacles:
                    plt.text(j, i, 'X', ha='center', va='center', fontsize=12, fontweight='bold')
                elif (i, j) == self.current_pos:
                    plt.text(j, i, 'A', ha='center', va='center', fontsize=12, fontweight='bold')
        
        plt.show()

# Create and visualize the environment
env = GridWorld(size=5)
print("🌍 Grid World Environment Created!")
print(f"Start: {env.start}, Goal: {env.goal}")
print(f"Obstacles: {env.obstacles}")
env.visualize()

## 2. Q-Learning Implementation

Complete Q-Learning agent with training and visualization capabilities.

In [ ]:
class QLearningAgent:
    def __init__(self, state_size, action_size, learning_rate=0.1, discount_factor=0.95, 
                 epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        
        # Initialize Q-table
        self.q_table = np.zeros((state_size, state_size, action_size))
        self.episode_rewards = []
        self.episode_steps = []
    
    def get_action(self, state, training=True):
        if training and np.random.random() < self.epsilon:
            return np.random.randint(self.action_size)
        else:
            return np.argmax(self.q_table[state[0], state[1]])
    
    def update_q_table(self, state, action, reward, next_state, done):
        current_q = self.q_table[state[0], state[1], action]
        
        if done:
            target_q = reward
        else:
            max_next_q = np.max(self.q_table[next_state[0], next_state[1]])
            target_q = reward + self.discount_factor * max_next_q
        
        self.q_table[state[0], state[1], action] += self.learning_rate * (target_q - current_q)
    
    def decay_epsilon(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

def train_agent(agent, env, episodes=1000):
    for episode in range(episodes):
        state = env.reset()
        total_reward = 0
        steps = 0
        
        for step in range(100):
            action = agent.get_action(state)
            next_state, reward, done = env.step(action)
            agent.update_q_table(state, action, reward, next_state, done)
            
            state = next_state
            total_reward += reward
            steps += 1
            
            if done:
                break
        
        agent.decay_epsilon()
        agent.episode_rewards.append(total_reward)
        agent.episode_steps.append(steps)
        
        if (episode + 1) % 200 == 0:
            avg_reward = np.mean(agent.episode_rewards[-100:])
            print(f"Episode {episode + 1}: Avg Reward = {avg_reward:.2f}, Epsilon = {agent.epsilon:.3f}")

# Train the agent
agent = QLearningAgent(state_size=5, action_size=4)
print("🚀 Training Q-Learning Agent...")
train_agent(agent, env, episodes=1000)
print("✅ Training Complete!")

## 3. Results Visualization

Analyze the learning progress and final policy.

In [ ]:
# Plot results
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Learning curve
axes[0, 0].plot(agent.episode_rewards, alpha=0.6)
moving_avg = pd.Series(agent.episode_rewards).rolling(window=50).mean()
axes[0, 0].plot(moving_avg, color='red', linewidth=2)
axes[0, 0].set_title('Learning Progress')
axes[0, 0].set_xlabel('Episode')
axes[0, 0].set_ylabel('Reward')
axes[0, 0].grid(True)

# Steps per episode
axes[0, 1].plot(agent.episode_steps, alpha=0.6)
moving_avg_steps = pd.Series(agent.episode_steps).rolling(window=50).mean()
axes[0, 1].plot(moving_avg_steps, color='red', linewidth=2)
axes[0, 1].set_title('Steps to Goal')
axes[0, 1].set_xlabel('Episode')
axes[0, 1].set_ylabel('Steps')
axes[0, 1].grid(True)

# Q-values heatmap
max_q_values = np.max(agent.q_table, axis=2)
im = axes[1, 0].imshow(max_q_values, cmap='viridis')
axes[1, 0].set_title('State Values')
plt.colorbar(im, ax=axes[1, 0])

# Policy visualization
policy = np.argmax(agent.q_table, axis=2)
arrows = ['↑', '→', '↓', '←']

axes[1, 1].imshow(env.grid, cmap='RdYlGn', alpha=0.3)
for i in range(env.size):
    for j in range(env.size):
        if (i, j) not in env.obstacles and (i, j) != env.goal:
            axes[1, 1].text(j, i, arrows[policy[i, j]], 
                           ha='center', va='center', fontsize=16, fontweight='bold')
        elif (i, j) == env.goal:
            axes[1, 1].text(j, i, 'G', ha='center', va='center', fontsize=16, fontweight='bold')
        elif (i, j) in env.obstacles:
            axes[1, 1].text(j, i, 'X', ha='center', va='center', fontsize=16, fontweight='bold')

axes[1, 1].set_title('Learned Policy')
axes[1, 1].set_xticks([])
axes[1, 1].set_yticks([])

plt.tight_layout()
plt.show()

print(f"Final Performance:")
print(f"Average reward (last 100 episodes): {np.mean(agent.episode_rewards[-100:]):.2f}")
print(f"Average steps (last 100 episodes): {np.mean(agent.episode_steps[-100:]):.1f}")